In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt



In [2]:
# Grid size
GRID_SIZE = 100
OBSTACLE_DENSITY = 0.2  # 20% of cells are obstacles

# Rewards
GOAL_REWARD = 100
STEP_PENALTY = -1
OBSTACLE_PENALTY = -10

# Define possible actions (up, down, left, right)
ACTIONS = [(0, 1), (1, 0), (0, -1), (-1, 0)]



In [3]:
# Initialize grid with obstacles
def create_grid():
    grid = np.zeros((GRID_SIZE, GRID_SIZE))
    num_obstacles = int(GRID_SIZE * GRID_SIZE * OBSTACLE_DENSITY)
    obstacles = random.sample([(i, j) for i in range(GRID_SIZE) for j in range(GRID_SIZE)], num_obstacles)
    for (i, j) in obstacles:
        grid[i, j] = OBSTACLE_PENALTY
    return grid



In [4]:
# Set start and goal points
def set_start_goal(grid):
    while True:
        start = (random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1))
        goal = (random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1))
        if start != goal and grid[start] != OBSTACLE_PENALTY and grid[goal] != OBSTACLE_PENALTY:
            grid[goal] = GOAL_REWARD
            return start, goal



In [5]:
# MDP model with Value Iteration
def value_iteration(grid, start, goal, gamma=0.9, theta=1e-4):
    value_grid = np.zeros_like(grid)
    policy_grid = np.zeros(grid.shape, dtype=int)
    while True:
        delta = 0
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if (i, j) == goal or grid[i, j] == OBSTACLE_PENALTY:
                    continue
                best_value = float('-inf')
                best_action = None
                for idx, action in enumerate(ACTIONS):
                    ni, nj = i + action[0], j + action[1]
                    if 0 <= ni < GRID_SIZE and 0 <= nj < GRID_SIZE:
                        reward = grid[ni, nj] if grid[ni, nj] != OBSTACLE_PENALTY else OBSTACLE_PENALTY
                        value = reward + gamma * value_grid[ni, nj]
                        if value > best_value:
                            best_value = value
                            best_action = idx
                delta = max(delta, abs(value_grid[i, j] - best_value))
                value_grid[i, j] = best_value
                policy_grid[i, j] = best_action
        if delta < theta:
            break
    return policy_grid



In [6]:
# Q-learning for the same environment
def q_learning(grid, start, goal, alpha=0.1, gamma=0.9, epsilon=0.1, episodes=1000):
    q_table = np.zeros((*grid.shape, len(ACTIONS)))
    for episode in range(episodes):
        state = start
        while state != goal:
            if random.uniform(0, 1) < epsilon:
                action_idx = random.randint(0, len(ACTIONS) - 1)
            else:
                action_idx = np.argmax(q_table[state[0], state[1]])
            action = ACTIONS[action_idx]
            next_state = (state[0] + action[0], state[1] + action[1])

            if 0 <= next_state[0] < GRID_SIZE and 0 <= next_state[1] < GRID_SIZE:
                reward = grid[next_state[0], next_state[1]]
                if reward == OBSTACLE_PENALTY:
                    reward = OBSTACLE_PENALTY  # Penalty for obstacle
                    next_state = state
                elif next_state == goal:
                    reward = GOAL_REWARD
                old_value = q_table[state[0], state[1], action_idx]
                next_max = np.max(q_table[next_state[0], next_state[1]])
                new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
                q_table[state[0], state[1], action_idx] = new_value
                state = next_state
    return q_table



In [7]:
# Benchmarking function
def benchmark(grid, policy_grid, q_table, start, goal):
    vi_path, ql_path = [start], [start]
    # Value Iteration Path
    state = start
    while state != goal:
        action_idx = policy_grid[state]
        action = ACTIONS[action_idx]
        state = (state[0] + action[0], state[1] + action[1])
        vi_path.append(state)
        if len(vi_path) > GRID_SIZE ** 2:
            break

    # Q-Learning Path
    state = start
    while state != goal:
        action_idx = np.argmax(q_table[state[0], state[1]])
        action = ACTIONS[action_idx]
        state = (state[0] + action[0], state[1] + action[1])
        ql_path.append(state)
        if len(ql_path) > GRID_SIZE ** 2:
            break

    return vi_path, ql_path



In [8]:
# Plotting function
def plot_paths(grid, vi_path, ql_path, start, goal):
    plt.imshow(grid, cmap='gray')
    plt.plot(*zip(*vi_path), label="Value Iteration Path", color="blue")
    plt.plot(*zip(*ql_path), label="Q-Learning Path", color="green")
    plt.scatter(*start[::-1], color="red", label="Start")
    plt.scatter(*goal[::-1], color="yellow", label="Goal")
    plt.legend()
    plt.show()



In [9]:
# Main execution
grid = create_grid()


In [10]:
start, goal = set_start_goal(grid)



In [11]:
policy_grid = value_iteration(grid, start, goal)


In [ ]:
q_table = q_learning(grid, start, goal)


In [ ]:
vi_path, ql_path = benchmark(grid, policy_grid, q_table, start, goal)
plot_paths(grid, vi_path, ql_path, start, goal)